In [ ]:
import collections
import os
import tempfile

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import urllib

import tensorflow as tf
import cv2    
import socket # 소켓 프로그래밍에 필요한 API를 제공하는 모듈
import struct # 바이트(bytes) 형식의 데이터 처리 모듈

import pickle # ﻿객체의 직렬화 및 역직렬화 지원 모듈﻿
import cv2 # OpenCV(실시간 이미지 프로세싱) 모듈
import time

#변수
toggle_data=b'\x00\x00\x00\x011'

#영상수신부 Socket (UDP)
ip = '172.30.1.52'
port = 50001
server_socket = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
server_socket.bind((ip, port))
print('영상수신 연결 대기')
data_buffer = b""
data_size = struct.calcsize("L")

#토글수신부 Socket (TCP/IP)
ip2 = '172.30.1.52'
port2 = 50003
tg_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
tg_socket.bind((ip2,port2))
print('토글수신 연결 대기')
tg_socket.listen(10)
tg_socket_ct, addr = tg_socket.accept()
print('토글수신 연결 완료')
tg_socket.setblocking(0)
tg_socket_ct.settimeout(0.1)

#frame송신부 Socket
ip3 = '172.30.1.52'
port3 = 50004 
fram_client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
fram_client_socket.connect((ip3,port3))

while True:
    # 영상 데이터 수신#######################################################
    while len(data_buffer) < data_size:
        data, addr = server_socket.recvfrom(65536)
        data_buffer += data
    packed_data_size = data_buffer[:data_size]
    data_buffer = data_buffer[data_size:]
    frame_size = struct.unpack(">L", packed_data_size)[0]

#     while len(data_buffer) < frame_size:
#         data_buffer += client_socket.recvfrom(65536)
    frame_data = data_buffer[:frame_size]
    data_buffer = data_buffer[frame_size:]
    #########################################################################
    
    #print("수신 프레임 크기 : {} bytes".format(frame_size) ,end="")
    #frame 직렬화 및 디코드
    #print("frame_data",frame_data)
    frame = pickle.loads(frame_data)
    #print("pic frame",frame)
    frame = cv2.imdecode(frame, cv2.COLOR_BGR2RGB)  
    #print("imdecode frame",frame)
    
    #만약 신호가 토글되면 frame하나를 포트 50004로 전송
    if toggle_data==b'\x00\x00\x00\x011':
        #print("전송 프레임 크기 : {} bytes".format(len(frame_data)),end="")
        fram_client_socket.sendall(struct.pack(">L", len(frame_data)) + frame_data)
        #print("토글 : ",toggle_data ,end="")
        #전송이후 toggle을 0으로 초기화
        toggle_data = b'0'
    
    #settimeout 기준으로 전송대기시간을 제한.
    try:
        toggle_data = tg_socket_ct.recv(8)
    except socket.timeout:
        pass
        
        
    #이미지 확인.
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1) & 0xFF
    if key== ord("q"):
        break



server_socket.close()
print('연결 종료')
cv2.destroyAllWindows()    

영상수신 연결 대기
토글수신 연결 대기
토글수신 연결 완료
